In [29]:
# этот блок нужен, если потребуется что то подгрузить через pip
# расчет на потенциальные модификации)

# общая идея решения записана в README в директории проекта

In [30]:
import json
import re
import csv

from collections import Counter

In [31]:
def load_dictionary(file_path: str, encod: str='utf-8') -> set:

    """Загрузка словаря с возможностью обработки разных кодировок"""

    # Изначально пытаемся открыть словарь в традиционном utf-8
    try:
        with open(file_path, 'r', encoding=encod) as f:
            # Приводим все слова к нижнему регистру для сокращения количества
            # разных символов и загружаем в множество для быстрого поиска
            return set(word.strip().lower() for word in f)

    # В случае, если utf-8 не обработает текс (такое может быть), выбираем
    # другую кодировку для русского языка
    except UnicodeDecodeError:
        encodings = ['cp1251', 'iso-8859-5', 'koi8-r']
        for enc in encodings:
            try:
                with open(file_path, 'r', encoding=enc) as f:
                    # Вернуть множество слов
                    return set(word.strip().lower() for word in f)

            except UnicodeDecodeError:
                continue

        # В крайнем случае выводим исключение, что текст нечитаем
        # (своеобразная защита от мусора)
        raise ValueError("Не удалось декодировать файл словаря")

In [32]:
def text_segmentation(text: str, dictionary: set, max_word_len: int=20) -> list:

    """Алгоритм сегментации текста"""

    n = len(text)

    # Ищем наилучшее разбиение, прменяя динамическое программирование
    # В dp будем хранить оценки оптимальности разбиения до позиции i (dp[i])
    dp = [-10**9] * (n + 1)
    dp[0] = 0
    prev = [-1] * (n + 1)

    # Заполняем таблицу оценок
    # Внешний цикл:
    # Перебирает все возможные начальные позиции разбиения
    # Пропускает недостижимые позиции (dp[i] == -10**9)
    # Внутренний цикл:
    # Рассматривает подстроки от i до j, где j меняется
    # от i+1 до min(n, i + max_word_len)
    # max_word_len ограничивает максимальную длину рассматриваемого слова

    # Решение основано на Принципе оптимальности Беллмана, которое
    # в контексте задачи формулируется как:
    # Оптимальное разбиение текста[0:n] содержит
    # оптимальное разбиение текста[0:k] для некоторого k < n.

    for i in range(n + 1):
        if dp[i] == -10**9:
            continue

        for j in range(i + 1, min(n, i + max_word_len) + 1):
            word = text[i:j].lower()
            # Начисляем "бонусы", если слово есть в словаре
            bonus = len(word)**2 if word in dictionary else 0
            # "Штраф", если слова неизвестны
            penalty = -0.5 * len(word) if word not in dictionary else 0

            # обновляем информацию о разбиении при нахождении решения
            if dp[j] < dp[i] + bonus + penalty:
                dp[j] = dp[i] + bonus + penalty
                prev[j] = i

    positions = []
    i = n

    # По сотавленной таблице пытаемся восстановить пробелы в тексте
    # Начинаем с конца текста (i = n)
    # Двигаемся назад по массиву prev
    # Для каждого перехода prev[i] → i добавляем позицию i в список
    # Исключаем позицию n (конец текста)
    # Разворачиваем список, так как мы шли с конца

    while i > 0:
        if prev[i] == -1:
            # Если не нашли разбиение, пропускаем один символ
            i -= 1
            continue

        if i != n:
            positions.append(i)

        i = prev[i]

    # Разворачиваем массив
    positions.reverse()

    # Вернуть список позиций пробела
    return positions

In [33]:
def process_file(input_file: str, output_file: str, dictionary_path: str):

    """Непостредственно обработка файла с текстом"""

    # Загружаем словарь
    dictionary = load_dictionary(dictionary_path)

    # Кодировки, которые могут использоваться при открытии файла
    encodings = ['utf-8', 'cp1251', 'iso-8859-5', 'koi8-r']

    # Пытаемся открыть разными кодировками
    for encod in encodings:
        try:
            with open(input_file, 'r', encoding=encod) as f:
                lines = f.readlines()
            break

        except UnicodeDecodeError:
            continue

    # в крайнем случае выводим исключение, что текст нечитаем
    # (снова защита от мусора)
    else:
        raise ValueError("Не удалось декодировать входной файл")

    # Обработка данных и запись результата в csv формат
    # Выходной файл записываем в utf-8
    with open(output_file, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['id', 'predicted_positions'])

        # Здесь проходимся по строчкам файла
        for line in lines[1:]:
            line = line.strip()
            # Разделяем ID и текст по первой запятой
            parts = line.split(',', 1)

            # Простейшая обработка пустых значений
            # На валидационном датасете не пригодится, но потенциально может
            # быть полезна
            if len(parts) < 2:
                continue

            id, text = parts
            # Прогоняем каждую строку через сегментатор
            positions = text_segmentation(text, dictionary)
            # Результат - пишем в файл - profit
            writer.writerow([id, json.dumps(positions)])

In [37]:
# Точка запуска
# Используем стандартную конструкцию, если хотим, чтобы наш код запускался как
# модуль где-нибудь еще (можно и без этого, но на всякий)
if __name__ == "__main__":
    valid_dataset = 'dataset_1937770_3.txt'
    output_dataset = 'output.csv'
    vacab = 'russian.txt'

    process_file(valid_dataset, output_dataset, vacab)